# Introduction

This notebook uses julia packages to perform causal set coordinate sprinking for schwarchild.  Similar procedures can be used to sprinkle other space-times.

## Constructing the Coordinate Density Probability Distribution

In [1]:
include("metric_definitions/shwarchild_spherical.jl")

metric_volume_density_jacobian (generic function with 2 methods)

In [2]:
schwar_4d = schwarchild_spherical()

metric_volume_density([1.6,2.0,3.141592653/2,0.0],schwar_4d)

4.0

In [3]:
using HCubature

lambda_bind = coords -> metric_volume_density(coords,schwar_4d)


PI = 3.141592653
eps = 1e-6
max_evaluations = typemax(Int)
a_tol = 0
coordinate_lower_bound = [0.0,1.0,0.0,0.0]
coordinate_upper_bound = [1.0,4.0,PI,2*PI]

metric_normalization, estimated_integral_error = hcubature(lambda_bind, coordinate_lower_bound, coordinate_upper_bound, rtol=sqrt(eps), atol=a_tol, maxevals=max_evaluations, initdiv=1)


(263.89388704371675, 0.04636251736351937)

In [4]:
volume_in_bounds = metric_normalization

263.89388704371675

In [5]:

coordinate_sampling_density = coords -> metric_volume_density(coords,schwar_4d)/metric_normalization



#11 (generic function with 1 method)

In [6]:
function_to_minimize = coords -> (-1) * (metric_volume_density(coords,schwar_4d)/metric_normalization)



#13 (generic function with 1 method)

In [7]:
# get the location of max values
using Optim


x0 = (coordinate_lower_bound + coordinate_upper_bound)/2.0

sol = optimize(function_to_minimize,coordinate_lower_bound,coordinate_upper_bound, x0)


 * Status: success

 * Candidate solution
    Final objective value:     -6.063043e-02

 * Found with
    Algorithm:     Fminbox with L-BFGS

 * Convergence measures
    |x - x'|               = 8.62e-08 ≰ 0.0e+00
    |x - x'|/|x'|          = 1.61e-08 ≰ 0.0e+00
    |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
    |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
    |g(x)|                 = 8.63e-11 ≤ 1.0e-08

 * Work counters
    Seconds run:   0  (vs limit Inf)
    Iterations:    3
    f(x) calls:    69
    ∇f(x) calls:   69


In [8]:
maximum_position = Optim.minimizer(sol)

max_probability_density = coordinate_sampling_density(maximum_position)

0.060630430581588776

In [9]:
max_probability_density,coordinate_sampling_density(x0)

(0.060630430581588776, 0.023683761946954925)

## Rejection Sampling

Here we begin the rejection sampling to obtain the sprinkled points

In [10]:
include("probability_utils/rejection_sampler.jl")

random_sampler_uniform (generic function with 2 methods)

In [12]:
random_sampler_uniform(coordinate_sampling_density,
                        coordinate_lower_bound,
                        coordinate_upper_bound,
                        max_probability_density,
                        10^2)



4-element Vector{Float64}:
 0.28953163226751455
 2.942059091086171
 1.8600702558978401
 5.619411896496509

In [13]:
# Set the scaling volume
ρ = 1
Number_of_points_in_volume = Int64(round(ρ * volume_in_bounds))

264

In [14]:

sprinkled_coordinates = zeros(Number_of_points_in_volume,4)


for sprinkle_ind in range(1,Number_of_points_in_volume) 
    sampled_coordinate = random_sampler_uniform(coordinate_sampling_density,
                                    coordinate_lower_bound,
                                    coordinate_upper_bound,
                                    max_probability_density,
                                    10^3)
    sprinkled_coordinates[sprinkle_ind,:] = sampled_coordinate
end

In [15]:
sprinkled_coordinates[1:3,:]

3×4 Matrix{Float64}:
 0.276179  2.0315   1.67998   1.87085
 0.641938  3.87068  2.27557   1.45416
 0.342561  3.37337  0.171359  4.42556